## Mounting drive

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## Importing necessary libraries

In [2]:
%%capture
%pip install datasets==1.18.3
%pip install transformers==4.17.0
%pip install jiwer

In [3]:
from huggingface_hub import notebook_login

notebook_login()

In [4]:
%%capture
!apt install git-lfs

## Prepare Data, Tokenizer, Feature Extractor

In [5]:
from datasets import load_dataset, load_metric

librispeech_train = load_dataset("librispeech_asr", "clean", split="train.100[:4000]")
librispeech_test = load_dataset("librispeech_asr", "clean", split="test")
librispeech_valid = load_dataset("librispeech_asr", "clean", split="validation")

In [6]:
#sample dataset
librispeech_train

Dataset({
    features: ['file', 'audio', 'text', 'speaker_id', 'chapter_id', 'id'],
    num_rows: 4000
})

In [7]:
librispeech_train = librispeech_train.remove_columns(["speaker_id", "chapter_id", "id"])
librispeech_test = librispeech_test.remove_columns(["speaker_id", "chapter_id", "id"])
librispeech_valid = librispeech_valid.remove_columns(["speaker_id", "chapter_id", "id"])

In [8]:
#sample dataset after removing columns
librispeech_train

Dataset({
    features: ['file', 'audio', 'text'],
    num_rows: 4000
})

In [9]:
from datasets import ClassLabel
import random
import pandas as pd
from IPython.display import display, HTML

def show_random_elements(dataset, num_examples=10):
    assert num_examples <= len(dataset), "Can't pick more elements than there are in the dataset."
    picks = []
    for _ in range(num_examples):
        pick = random.randint(0, len(dataset)-1)
        while pick in picks:
            pick = random.randint(0, len(dataset)-1)
        picks.append(pick)

    df = pd.DataFrame(dataset[picks])
    display(HTML(df.to_html()))

In [10]:
show_random_elements(librispeech_train.remove_columns(["audio", "file"]), num_examples=10)

,text
0,AS MATTERS BETWEEN PRIVATE INDIVIDUALS CAN BE DECIDED DURING THE LAST CENTURY AT LEAST HALF OF THE WARS THAT HAVE BEEN FOUGHT HAVE BEEN CIVIL AND NOT FOREIGN WARS THERE ARE BIG AND POWERFUL NATIONS WHICH HABITUALLY COMMIT EITHER UPON OTHER NATIONS
1,ONE OF THEM SAID IT WAS WORTH NINETEEN HUNDRED DOLLARS TO HAVE HEARD THE TALK THAT WENT ON IN THE COURT ROOM LATER MISTER WHISTLER RECEIVED MUCH MORE THAN TWO HUNDRED GUINEAS FOR A SINGLE PICTURE
2,AND ENJOYED THE LIFE OF TRAVEL VERY MUCH BUT THEY WERE ALWAYS GLAD WHEN THE NIGHT CAME AND THEY STOPPED FOR THEIR RESTING TIME THEN THE DOCTOR USED TO MAKE A LITTLE FIRE OF STICKS
3,AN DAT MINES ME A OWL WHAR TURNT HISSE'F INTER A MAN AN EF A OWL KIN DO DAT
4,TO LIVE BY HIS OWN INDUSTRY IN ENGLAND RATHER THAN ON THE INDUSTRY OF THE OVERLADEN PEOPLE OF FRANCE WHAT PROOF HAD HE OF THIS HE HANDED IN THE NAMES OF TWO WITNESSES THEOPHILE GABELLE AND ALEXANDRE MANETTE
5,THAT IS WHY I HAVE TO FLY AWAY SOUTH AS SOON AS THE GROUND FREEZES AT ALL IT'S WONDERFUL SIGHED PETER I DON'T SUPPOSE ANY ONE ELSE CAN FIND HIDDEN WORMS THAT WAY MY COUSIN JACK SNIPE CAN REPLIED LONGBILL PROMPTLY
6,TWO GUINEAS ROARED MERRY SHAKING IT AT SILVER
7,THAT DURING A HEAVY SHOWER JULY THIRD EIGHTEEN SIXTY HE HEARD A PECULIAR SOUND AT HIS FEET AND LOOKING DOWN SAW A SNAKE LYING AS IF STUNNED BY A FALL IT THEN CAME TO LIFE GRAY SNAKE ABOUT A FOOT LONG
8,THE ONLY EXPENSE EXCEPT THAT OF BOOKS TO WHICH A HOUSE BOARDER WAS THEN SUBJECT WAS THE FEE TO A TUTOR AMOUNTING I THINK TO TEN GUINEAS MY TUTOR TOOK ME WITHOUT THE FEE
9,FOR THAT MACHAERUS WOULD NOT BE SUCH AN ASSISTANT AS HE COULD DEPEND ON AS IT APPEARED BY WHAT HE HAD DONE ALREADY


In [11]:
import re
chars_to_ignore_regex = '[\,\?\.\!\-\;\:\"]'

def remove_special_characters(batch):
    batch["text"] = re.sub(chars_to_ignore_regex, '', batch["text"]).lower() + " "
    return batch

In [12]:
librispeech_train = librispeech_train.map(remove_special_characters)
librispeech_test = librispeech_test.map(remove_special_characters)
librispeech_valid = librispeech_valid.map(remove_special_characters)

0ex [00:00, ?ex/s]

0ex [00:00, ?ex/s]

0ex [00:00, ?ex/s]

In [13]:
show_random_elements(librispeech_train.remove_columns(["audio", "file"]))

,text
0,it doesn't taste a bit like hers i should think marilla's raspberry cordial would prob'ly be much nicer than missus lynde's said anne loyally marilla is a famous cook
1,oh marilla i know by your face that it's been no use she said sorrowfully missus barry won't forgive me missus barry indeed snapped marilla of all the unreasonable women i ever saw she's the worst
2,clavichord its box like case was first placed on a table later on its own stand and increased in elegance not until the eighteenth century was each key provided with a separate string
3,although because of the wonderful athletic prowess of its students they have met and defeated the athletes of many a white university on the football field
4,you talk as if it were a real farm instead of a place no one would have as a gift uncle john looked sober again anyone live on the place major he inquired
5,and according to your argument they are also justified it is a matter of plane he said airily viewed from my standpoint they are sordid criminals the sort of person that t x meets i presume in the course of his daily work
6,they were habitually indifferent to self exaltation and allowed themselves to be thrust into this or that unfitting role professing that the queen's government and the good of the country were their only considerations
7,yes said jerry which would make it over there that the thing showed up let's take a look at the ground and see if will was dreaming or not before half a minute had passed frank was pointing to certain marks plainly seen in the inch and more of snow that had fallen on the previous night
8,a face that hid much and revealed little it told of power and intellect but the soul of the man was a hidden thing
9,that is that inheritance and succession will last for ever whoever may be the recognized heir but the saint simonians wish the heir to be designated by the magistrate


In [14]:
def extract_all_chars(batch):
  all_text = " ".join(batch["text"])
  vocab = list(set(all_text))
  return {"vocab": [vocab], "all_text": [all_text]}

In [15]:
vocab_train = librispeech_train.map(extract_all_chars, batched=True, batch_size=-1, keep_in_memory=True, remove_columns=librispeech_train.column_names)
vocab_test = librispeech_test.map(extract_all_chars, batched=True, batch_size=-1, keep_in_memory=True, remove_columns=librispeech_test.column_names)
vocab_dev = librispeech_valid.map(extract_all_chars, batched=True, batch_size=-1, keep_in_memory=True, remove_columns=librispeech_valid.column_names)

  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

In [16]:
vocab_list = list(set(vocab_train["vocab"][0]) | set(vocab_dev["vocab"][0]) | set(vocab_test["vocab"][0]) )

In [17]:
vocab_dict = {v: k for k, v in enumerate(vocab_list)}
vocab_dict

{'h': 0,
 'q': 1,
 ' ': 2,
 'u': 3,
 'f': 4,
 'a': 5,
 'x': 6,
 'k': 7,
 'v': 8,
 'd': 9,
 'y': 10,
 'b': 11,
 'm': 12,
 'g': 13,
 'w': 14,
 'p': 15,
 't': 16,
 's': 17,
 'z': 18,
 'l': 19,
 'n': 20,
 'e': 21,
 'j': 22,
 "'": 23,
 'i': 24,
 'r': 25,
 'o': 26,
 'c': 27}

In [18]:
vocab_dict["|"] = vocab_dict[" "]
del vocab_dict[" "]

In [19]:
vocab_dict["[UNK]"] = len(vocab_dict)
vocab_dict["[PAD]"] = len(vocab_dict)
len(vocab_dict)

30

In [20]:
import json
with open('vocab_SST_medium_accuracy.json', 'w') as vocab_file:
    json.dump(vocab_dict, vocab_file)

In [21]:
from transformers import Wav2Vec2CTCTokenizer

tokenizer = Wav2Vec2CTCTokenizer("./vocab_SST_medium_accuracy.json", unk_token="[UNK]", pad_token="[PAD]", word_delimiter_token="|")

### Create Wav2Vec2 Feature Extractor

In [22]:
from transformers import Wav2Vec2FeatureExtractor

feature_extractor = Wav2Vec2FeatureExtractor(feature_size=1, sampling_rate=16000, padding_value=0.0, do_normalize=True, return_attention_mask=False)

In [23]:
from transformers import Wav2Vec2Processor

processor = Wav2Vec2Processor(feature_extractor=feature_extractor, tokenizer=tokenizer)

## Saving tokenizer

In [24]:
tokenizer.save_pretrained("/content/drive/My Drive/Capstone-Lisa-Mimi")

('/content/drive/My Drive/Capstone-Lisa-Mimi/tokenizer_config.json',
 '/content/drive/My Drive/Capstone-Lisa-Mimi/special_tokens_map.json',
 '/content/drive/My Drive/Capstone-Lisa-Mimi/vocab.json',
 '/content/drive/My Drive/Capstone-Lisa-Mimi/added_tokens.json')

## Preprocess data

In [25]:
import numpy as np
np.object = np.object_


def prepare_dataset(batch):
    audio = batch["audio"]
    audio_array = np.array(audio["array"], dtype=np.float32)

    # Batched output is "un-batched" to ensure mapping is correct
    batch["input_values"] = processor(audio_array, sampling_rate=audio["sampling_rate"]).input_values[0]
    batch["input_length"] = len(batch["input_values"])

    with processor.as_target_processor():
        batch["labels"] = processor(batch["text"]).input_ids

    return batch

In [26]:
librispeech_train = librispeech_train.map(prepare_dataset, remove_columns=librispeech_train.column_names)
librispeech_valid= librispeech_valid.map(prepare_dataset, remove_columns=librispeech_valid.column_names)
librispeech_test = librispeech_test.map(prepare_dataset, remove_columns=librispeech_test.column_names)

0ex [00:00, ?ex/s]

0ex [00:00, ?ex/s]

0ex [00:00, ?ex/s]

In [27]:
max_input_length_in_sec = 4.0
train_dataset= librispeech_train.filter(lambda x: x < max_input_length_in_sec * processor.feature_extractor.sampling_rate, input_columns=["input_length"])

  0%|          | 0/4 [00:00<?, ?ba/s]

### Set-up Trainer

In [28]:
import torch

from dataclasses import dataclass, field
from typing import Any, Dict, List, Optional, Union

@dataclass
class DataCollatorCTCWithPadding:
    """
    Data collator that will dynamically pad the inputs received.
    Args:
        processor (:class:`~transformers.Wav2Vec2Processor`)
            The processor used for proccessing the data.
        padding (:obj:`bool`, :obj:`str` or :class:`~transformers.tokenization_utils_base.PaddingStrategy`, `optional`, defaults to :obj:`True`):
            Select a strategy to pad the returned sequences (according to the model's padding side and padding index)
            among:
            * :obj:`True` or :obj:`'longest'`: Pad to the longest sequence in the batch (or no padding if only a single
              sequence if provided).
            * :obj:`'max_length'`: Pad to a maximum length specified with the argument :obj:`max_length` or to the
              maximum acceptable input length for the model if that argument is not provided.
            * :obj:`False` or :obj:`'do_not_pad'` (default): No padding (i.e., can output a batch with sequences of
              different lengths).
    """

    processor: Wav2Vec2Processor
    padding: Union[bool, str] = True

    def __call__(self, features: List[Dict[str, Union[List[int], torch.Tensor]]]) -> Dict[str, torch.Tensor]:
        # split inputs and labels since they have to be of different lenghts and need
        # different padding methods
        input_features = [{"input_values": feature["input_values"]} for feature in features]
        label_features = [{"input_ids": feature["labels"]} for feature in features]

        batch = self.processor.pad(
            input_features,
            padding=self.padding,
            return_tensors="pt",
        )
        with self.processor.as_target_processor():
            labels_batch = self.processor.pad(
                label_features,
                padding=self.padding,
                return_tensors="pt",
            )

        # replace padding with -100 to ignore loss correctly
        labels = labels_batch["input_ids"].masked_fill(labels_batch.attention_mask.ne(1), -100)

        batch["labels"] = labels

        return batch

In [29]:
data_collator = DataCollatorCTCWithPadding(processor=processor, padding=True)

In [30]:
wer_metric = load_metric("wer")

In [31]:
def compute_metrics(pred):
    pred_logits = pred.predictions
    pred_ids = np.argmax(pred_logits, axis=-1)

    pred.label_ids[pred.label_ids == -100] = processor.tokenizer.pad_token_id

    pred_str = processor.batch_decode(pred_ids)
    # we do not want to group tokens when computing the metrics
    label_str = processor.batch_decode(pred.label_ids, group_tokens=False)

    wer = wer_metric.compute(predictions=pred_str, references=label_str)

    return {"wer": wer}

In [32]:
from transformers import Wav2Vec2ForCTC

model = Wav2Vec2ForCTC.from_pretrained(
    "facebook/wav2vec2-base",
    ctc_loss_reduction="mean",
    pad_token_id=processor.tokenizer.pad_token_id,
)

/usr/local/lib/python3.10/dist-packages/transformers/configuration_utils.py:356: UserWarning: Passing `gradient_checkpointing` to a config initialization is deprecated and will be removed in v5 Transformers. Using `model.gradient_checkpointing_enable()` instead, or if you are using the `Trainer` API, pass `gradient_checkpointing=True` in your `TrainingArguments`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/nn/utils/weight_norm.py:28: UserWarning: torch.nn.utils.weight_norm is deprecated in favor of torch.nn.utils.parametrizations.weight_norm.
  warnings.warn("torch.nn.utils.weight_norm is deprecated in favor of torch.nn.utils.parametrizations.weight_norm.")
Some weights of the model checkpoint at facebook/wav2vec2-base were not used when initializing Wav2Vec2ForCTC: ['quantizer.weight_proj.weight', 'project_hid.weight', 'project_q.weight', 'project_hid.bias', 'project_q.bias', 'quantizer.weight_proj.bias', 'quantizer.codevectors']
- This IS expected if you are initia

In [33]:
model.freeze_feature_encoder()

In [34]:
from transformers import TrainingArguments

training_args = TrainingArguments(
  output_dir="/content/drive/My Drive/Capstone-Lisa-Mimi",
  group_by_length=True,
  per_device_train_batch_size=32,
  per_device_eval_batch_size=32,
  evaluation_strategy="steps",
  num_train_epochs=10,
  fp16=True,
  gradient_checkpointing=True,
  save_steps=500,
  eval_steps=500,
  logging_steps=500,
  learning_rate=1e-4,
  weight_decay=0.005,
  warmup_steps=1000,
  save_total_limit=2,
)

In [35]:
from transformers import Trainer

trainer = Trainer(
    model=model,
    data_collator=data_collator,
    args=training_args,
    compute_metrics=compute_metrics,
    train_dataset=librispeech_train,
    eval_dataset=librispeech_valid,
    tokenizer=processor.feature_extractor,
)

Using amp half precision backend


## Training

In [36]:
np.bool = np.bool_
trainer.train()

The following columns in the training set  don't have a corresponding argument in `Wav2Vec2ForCTC.forward` and have been ignored: input_length. If input_length are not expected by `Wav2Vec2ForCTC.forward`,  you can safely ignore this message.
/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 4000
  Num Epochs = 10
  Instantaneous batch size per device = 32
  Total train batch size (w. parallel, distributed & accumulation) = 32
  Gradient Accumulation steps = 1
  Total optimization steps = 1250
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:464: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.4 we will raise an

Step,Training Loss,Validation Loss,Wer
500,3.633600,2.677950,0.999926
1000,0.403200,0.255734,0.204717


The following columns in the evaluation set  don't have a corresponding argument in `Wav2Vec2ForCTC.forward` and have been ignored: input_length. If input_length are not expected by `Wav2Vec2ForCTC.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 2703
  Batch size = 32
Saving model checkpoint to /content/drive/My Drive/Capstone-Lisa-Mimi/checkpoint-500
Configuration saved in /content/drive/My Drive/Capstone-Lisa-Mimi/checkpoint-500/config.json
Model weights saved in /content/drive/My Drive/Capstone-Lisa-Mimi/checkpoint-500/pytorch_model.bin
Feature extractor saved in /content/drive/My Drive/Capstone-Lisa-Mimi/checkpoint-500/preprocessor_config.json
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:464: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.4 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to pre

TrainOutput(global_step=1250, training_loss=1.6332405075073242, metrics={'train_runtime': 4585.4751, 'train_samples_per_second': 8.723, 'train_steps_per_second': 0.273, 'total_flos': 4.81607620176937e+18, 'train_loss': 1.6332405075073242, 'epoch': 10.0})

In [37]:
trainer.save_model("/content/drive/MyDrive/Capstone-Lisa-Mimi")

Saving model checkpoint to /content/drive/MyDrive/Capstone-Lisa-Mimi
Configuration saved in /content/drive/MyDrive/Capstone-Lisa-Mimi/config.json
Model weights saved in /content/drive/MyDrive/Capstone-Lisa-Mimi/pytorch_model.bin
Feature extractor saved in /content/drive/MyDrive/Capstone-Lisa-Mimi/preprocessor_config.json


## Saving model

In [38]:
# saving model on my drive
model = Wav2Vec2ForCTC.from_pretrained("/content/drive/MyDrive/Capstone-Lisa-Mimi")
processor = Wav2Vec2Processor.from_pretrained("/content/drive/MyDrive/Capstone-Lisa-Mimi")

# Define the path where you want to save the final model
save_directory = "/content/drive/MyDrive/Capstone-Lisa-Mimi/model"

# Save the model and processor
model.save_pretrained(save_directory)
processor.save_pretrained(save_directory)

print(f"Model and processor saved in {save_directory}")

loading configuration file /content/drive/MyDrive/Capstone-Lisa-Mimi/config.json
Model config Wav2Vec2Config {
  "_name_or_path": "facebook/wav2vec2-base",
  "activation_dropout": 0.0,
  "adapter_kernel_size": 3,
  "adapter_stride": 2,
  "add_adapter": false,
  "apply_spec_augment": true,
  "architectures": [
    "Wav2Vec2ForCTC"
  ],
  "attention_dropout": 0.1,
  "bos_token_id": 1,
  "classifier_proj_size": 256,
  "codevector_dim": 256,
  "contrastive_logits_temperature": 0.1,
  "conv_bias": false,
  "conv_dim": [
    512,
    512,
    512,
    512,
    512,
    512,
    512
  ],
  "conv_kernel": [
    10,
    3,
    3,
    3,
    3,
    2,
    2
  ],
  "conv_stride": [
    5,
    2,
    2,
    2,
    2,
    2,
    2
  ],
  "ctc_loss_reduction": "mean",
  "ctc_zero_infinity": false,
  "diversity_loss_weight": 0.1,
  "do_stable_layer_norm": false,
  "eos_token_id": 2,
  "feat_extract_activation": "gelu",
  "feat_extract_norm": "group",
  "feat_proj_dropout": 0.1,
  "feat_quantizer_drop

Model and processor saved in /content/drive/MyDrive/Capstone-Lisa-Mimi/model


## Evaluation

In [39]:
from transformers import AutoModelForCTC, Wav2Vec2Processor

model = AutoModelForCTC.from_pretrained("/content/drive/MyDrive/Capstone-Lisa-Mimi/model")
processor = Wav2Vec2Processor.from_pretrained("/content/drive/MyDrive/Capstone-Lisa-Mimi/model")

loading configuration file /content/drive/MyDrive/Capstone-Lisa-Mimi/model/config.json
Model config Wav2Vec2Config {
  "_name_or_path": "/content/drive/MyDrive/Capstone-Lisa-Mimi/model",
  "activation_dropout": 0.0,
  "adapter_kernel_size": 3,
  "adapter_stride": 2,
  "add_adapter": false,
  "apply_spec_augment": true,
  "architectures": [
    "Wav2Vec2ForCTC"
  ],
  "attention_dropout": 0.1,
  "bos_token_id": 1,
  "classifier_proj_size": 256,
  "codevector_dim": 256,
  "contrastive_logits_temperature": 0.1,
  "conv_bias": false,
  "conv_dim": [
    512,
    512,
    512,
    512,
    512,
    512,
    512
  ],
  "conv_kernel": [
    10,
    3,
    3,
    3,
    3,
    2,
    2
  ],
  "conv_stride": [
    5,
    2,
    2,
    2,
    2,
    2,
    2
  ],
  "ctc_loss_reduction": "mean",
  "ctc_zero_infinity": false,
  "diversity_loss_weight": 0.1,
  "do_stable_layer_norm": false,
  "eos_token_id": 2,
  "feat_extract_activation": "gelu",
  "feat_extract_norm": "group",
  "feat_proj_dropou

In [40]:
final_model = model.cuda()

In [41]:
final_model

Wav2Vec2ForCTC(
  (wav2vec2): Wav2Vec2Model(
    (feature_extractor): Wav2Vec2FeatureEncoder(
      (conv_layers): ModuleList(
        (0): Wav2Vec2GroupNormConvLayer(
          (conv): Conv1d(1, 512, kernel_size=(10,), stride=(5,), bias=False)
          (activation): GELUActivation()
          (layer_norm): GroupNorm(512, 512, eps=1e-05, affine=True)
        )
        (1-4): 4 x Wav2Vec2NoLayerNormConvLayer(
          (conv): Conv1d(512, 512, kernel_size=(3,), stride=(2,), bias=False)
          (activation): GELUActivation()
        )
        (5-6): 2 x Wav2Vec2NoLayerNormConvLayer(
          (conv): Conv1d(512, 512, kernel_size=(2,), stride=(2,), bias=False)
          (activation): GELUActivation()
        )
      )
    )
    (feature_projection): Wav2Vec2FeatureProjection(
      (layer_norm): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
      (projection): Linear(in_features=512, out_features=768, bias=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder)

In [42]:
def map_to_result(batch):
  with torch.no_grad():
    input_values = torch.tensor(batch["input_values"], device="cuda").unsqueeze(0)
    logits = model(input_values).logits

  pred_ids = torch.argmax(logits, dim=-1)
  batch["pred_str"] = processor.batch_decode(pred_ids)[0]
  batch["text"] = processor.decode(batch["labels"], group_tokens=False)

  return batch

In [45]:
results = librispeech_test.map(map_to_result, remove_columns=librispeech_test.column_names)

0ex [00:00, ?ex/s]

/usr/local/lib/python3.10/dist-packages/torch/nn/modules/conv.py:306: UserWarning: Plan failed with a cudnnException: CUDNN_BACKEND_EXECUTION_PLAN_DESCRIPTOR: cudnnFinalize Descriptor Failed cudnn_status: CUDNN_STATUS_NOT_SUPPORTED (Triggered internally at ../aten/src/ATen/native/cudnn/Conv_v8.cpp:919.)
  return F.conv1d(input, weight, bias, self.stride,


In [46]:
print("Test WER: {:.3f}".format(wer_metric.compute(predictions=results["pred_str"], references=results["text"])))

Test WER: 0.120


In [47]:
show_random_elements(results)

,pred_str,text
0,a word should now be said about the origin of luther's commentary angellations,a word should now be said about the origin of luther's commentary on galatians
1,you have received us with all that courtecy and hospitality for which your character in england stands so high,you have received us with all that courtesy and hospitality for which your character in england stands so high
2,so we harried the coast of norway,so we harried the coast of norway
3,for many then this book has been a source of fascination surely one of the most influential novels ever written an inspiration for such sientists and discoverers as engineers symon lake ocheographer william bebee poler traveller sare earnest schackleton,for many then this book has been a source of fascination surely one of the most influential novels ever written an inspiration for such scientists and discoverers as engineer simon lake oceanographer william beebe polar traveler sir ernest shackleton
4,tabby had tended them in their childhood they and none other should tend her in her infirmity and age,tabby had tended them in their childhood they and none other should tend her in her infirmity and age
5,mistress fits outh had been carried off by the shareaf's daughter and her maids as soon as they had entered the house so that robbin alone had the care of mount fiche,mistress fitzooth had been carried off by the sheriff's daughter and her maids as soon as they had entered the house so that robin alone had the care of montfichet
6,said captain donniythorne seating himself where he could see along the short passage to the open dary door,said captain donnithorne seating himself where he could see along the short passage to the open dairy door
7,isn't he splendid cried jasper an intense pride swelling up father knew how to do it,isn't he splendid cried jasper in intense pride swelling up father knew how to do it
8,their diving stone poised on its rude supports and rocking under their plunges and the rough humne stones of the sloping break water over which they scrambled in their hoarse play gleamed with cold wet luster,their diving stone poised on its rude supports and rocking under their plunges and the rough hewn stones of the sloping breakwater over which they scrambled in their horseplay gleamed with cold wet lustre
9,see said oncus pointing north and south at the evident marks of the broad trail on either side of him the dark haire has gone toward the forest,see said uncas pointing north and south at the evident marks of the broad trail on either side of him the dark hair has gone toward the forest
